# decorator
- 함수를 수정하지 않고도 함수에 특정 동작을 추가하거나 작동방식을 바꿀 수 있음



##### 함수를
- 일반 값처럼 변수에 저장될 수 있고,
-  다른 함수의 인자로 넘기거나, 
-  리턴값으로 사용 가능

## 1. decorator 기본 지식
- 다른 함수를 인자로 받은 콜러블(데커레이트된 함수)
- 데커레이트된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 함수를 다른 함수나 콜러블 객체로 대체한다.

In [ ]:
@decorate
def target():
    print('running target()')

In [ ]:
def target():
    print('running target()')

target = decorator(target)   # target 변수에 함수를 재할당

- 상기 두 코드의 결과는 동일하다
- 두 코드를 실행한 후, target은 원래의 target()함수를 가리키는 것이 아니라, decorate(target)이 반환된 함수를가리킨다.

### 1) 일반적으로 데커레이터는 함수를 다른 함수로 대체한다.

In [5]:
def deco(func):
    def inner():
        print('running inner()')
    return inner                    # deco()는 inner 함수 객체를 반환

@deco
def target1():                        # target()를 deco로 데커레이트
    print('running target1()')


print('target1():')
print( target1())                    # 데커레이트된 target()을 호출하면 실제로 inner를 실행
print('target1:')
print( target1)                      # target이 inner()를 가리키고 있음



target1():
running inner()
None
target1:
<function deco.<locals>.inner at 0x000002B8C6F83010>


In [7]:
def target2():
    print('running target2()')

target2 = deco(target2())            # @deco
                                     # def target2():
                                     #      print('running target2()') 와 동일한 결과

print('target2():')
print( target2())                    # 데커레이트된 target2()을 호출하면 실제로 inner를 실행
print('target2:')
print( target2)                      # target2이 inner()를 가리키고 있음


running target2()
target2():
running inner()
None
target2:
<function deco.<locals>.inner at 0x000002B8C6D13760>


##### 데커레이터
-  다른 함수를 인수로 전달해서 호출 <p>
-  데커레이터 된 함수를 다른 함수로 대체 가능<p>
-  모듈이 로딩될 때 바로 실행<p>

### 2) 데코레이터로 원래 함수의 인자 그대로 넘기기 : *args, **kwargs 사용

In [8]:
@decorater
def say(msg):
    print(msg)

msg = '안녕하세요'
say(msg)

NameError: name 'decorater' is not defined

- decorater 안의 wrapper()함수가 msg 인자를 무시했기 때문에 오류 발생
- 원래 함수의 인자를 decorator 내부 함수로 넘기려면 *args와 **kwargs 사용해야 함

In [9]:
def decorate(func):
    def wrapper(*args, **kwars):
        print('before')
        func(*args, **kwars)
        print('after')
    return wrapper

In [10]:
@decorate
def say(msg):
    print(msg)

msg = '안녕하세요'
say(msg)

before
안녕하세요
after


### 3)  데코레이터로부터 원래 함수의 리턴값 그대로 받기

In [25]:
@decorate
def give_hi():
    return "Hi"

result = give_hi()
print(result)

before
after
None


- 원래 함수의 'Hi'가 데코레이터에서는 리턴안됨
- wrapper()함수에서 원래 리턴값을 보존해주지 않기 때문
- -> 원래 함수의 리턴값을 변수에 저장해주기

In [30]:
def decorater(func):
    def wrapper(*args, **kwargs):
        print('before')
        value = func(*args, ** kwargs)
        print('after')
        return value
    return wrapper

@decorater
def give_hi():
    return 'Hi'

result = give_hi()


before
after


In [31]:
print(result)

Hi


## 2. 파이썬이 데커레이터를 실행하는 시점

##### * 데커레이터
- 데커레이트된 함수가 정의된 직후 실행
- 파이썬이 모듈을 실해하는 시점, 즉 임포트 타임에 실행

In [13]:
registry = []                              # registry : @register로 데커레이트된 함수들에 대한 참조 담는 list

def register(func):                        # 함수를 인수로 받음
    print('running register(%s'%func)      # 데커레이트된 함수 출력
    registry.append(func)           
    return func                            # func 함수를 반환 (여기서는 인수로 받은 함수를 그대로 반환)

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__== '__main__':
    main()

running register(<function f1 at 0x000002B8C7363D00>
running register(<function f2 at 0x000002B8C7363010>
running main()
registry -> [<function f1 at 0x000002B8C7363D00>, <function f2 at 0x000002B8C7363010>]
running f1()
running f2()
running f3()


##### 함수 register() 
- register()은 제일 먼저 2번 실행.-> import time
- 모듈이 임포트되자마자 실행
  
##### 데커레이트된 함수 f1(), f2() 
- 명시적으로  호출될 때만 실행됨-> 런타임(runtime)

##### Reference
- [파이썬] 데코레이터 기본 사용법 : https://www.daleseo.com/python-decorators/
- 전문가를 위한 파이썬(한빛미디어, 루시아누 하말류, 2016)